# Importing Importanta packages

In [ ]:
# Install the torchmetrics package for storing loss, evaluation metrics, etc.
!pip install lightning-utilities


In [ ]:
!pip install torchmetrics --no-deps

In [ ]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision.models as models # Import models module
from torchvision import transforms
import torchvision
from torchvision.transforms import v2
from torchvision.models import EfficientNet_B0_Weights
from torchvision.transforms.functional import to_pil_image
from torchmetrics import MeanMetric, Accuracy
from torchmetrics import ConfusionMatrix, Accuracy, Precision, Recall, F1Score

In [ ]:
# Make sure to change runtime to GPU
# Check if GPU is avaiable
device = "cuda" if torch.cuda.is_available() \
          else "mps" if torch.mps.is_available() \
          else "cpu"
print("Device:", device)

# Getting The Data

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("farjanakabirsamanta/skin-cancer-dataset")

print("Path to dataset files:", path)

In [ ]:

# List downloaded files
print("Dataset contents:", os.listdir(path))

In [ ]:

# Checking for any subdirectories inside the downloaded dataset path
print("Subdirectories:", [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))])

In [ ]:

# Update path to the correct subdirectory
dataset_path = os.path.join(path, "Skin Cancer")

# List contents in the new path
print("Dataset contents:", os.listdir(dataset_path))

In [ ]:
import glob


In [ ]:

# Look for images inside the "Skin Cancer" directory
image_files = glob.glob(os.path.join(dataset_path, "*.jpg"))
print(f"Total images found: {len(image_files)}")

In [ ]:
#I can't find where the images of dataset so I will dig more to see if there is more subfolders.

In [ ]:
# List subdirectories within "Skin Cancer"
print("Subdirectories inside 'Skin Cancer':", [f for f in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, f))])


In [ ]:
# Update dataset path to point to the actual files
dataset_path = os.path.join(path, "Skin Cancer", "Skin Cancer")

# List the files inside the correct dataset folder
print("Updated dataset contents:", os.listdir(dataset_path))

In [ ]:
# Look for images inside the correct dataset folder
image_files = glob.glob(os.path.join(dataset_path, "*.jpg"))
print(f"Total images found: {len(image_files)}")

In [ ]:
# Load metadata CSV
# The original path was incorrect. Adjust to find the correct location of the metadata file.
# The metadata file might be in the base directory of the dataset, not in the 'Skin Cancer/Skin Cancer' subfolder.
metadata_path = os.path.join(path, "HAM10000_metadata.csv")
df = pd.read_csv(metadata_path)

# Attach correct file paths to images
df['filepath'] = df['image_id'].apply(lambda x: os.path.join(dataset_path, f"{x}.jpg"))

# Verify if all image files exist
missing_files = df['filepath'].apply(lambda x: not os.path.exists(x)).sum()
print(f"Missing images: {missing_files}")

# Check dataset distribution
print(df['dx'].value_counts())

In [ ]:
# Get unique classes
classes = df['dx'].unique()

# Create a figure
fig, axes = plt.subplots(2, 4, figsize=(15, 6))  # 2 rows, 4 columns

for ax, class_name in zip(axes.flat, classes):
    # Select a random image from this class
    sample = df[df['dx'] == class_name].sample(1)
    img_path = sample['filepath'].values[0]

    # Load and display the image
    img = Image.open(img_path)
    ax.imshow(img)
    ax.set_title(class_name)
    ax.axis('off')  # Hide axes

plt.tight_layout()
plt.show()

In [ ]:
num_class = len(classes)
print("Number of classes:", num_class)

# Applying Data Augmentation

In [ ]:
# data augmentation for train
# --- build transforms using the weights’ recommended preprocessing ---
weights = EfficientNet_B0_Weights.IMAGENET1K_V1  # NEW

train_transforms = v2.Compose([
    v2.RandomResizedCrop(224, scale=(0.85, 1.0)),  # CHANGED: EfficientNet-B0 is 224x224
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomVerticalFlip(p=0.1),
    v2.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.15, hue=0.02),
    v2.ToImage(), # Convert to tensor before normalization
    v2.ToDtype(torch.float32, scale=True), # Convert to float32 before normalization
    v2.Normalize(mean=weights.transforms().mean, std=weights.transforms().std),  # CHANGED
])

val_transforms = v2.Compose([
    v2.Resize(int(224 * 1.15)),
    v2.CenterCrop(224),
    v2.ToImage(), # Convert to tensor before normalization
    v2.ToDtype(torch.float32, scale=True), # Convert to float32 before normalization
    v2.Normalize(mean=weights.transforms().mean, std=weights.transforms().std),  # CHANGED
])

test_transforms = val_transforms  # keep same as validation

In [ ]:
# data augmentation for validation and test
eval_transform = v2.Compose([
    v2.Resize((224, 224)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=weights.transforms().mean,
                 std=weights.transforms().std),
])



In [ ]:
print(os.listdir(dataset_path))


# Organizing the folders

In [ ]:
import shutil

In [ ]:
#My Dataset was not organized by class so I have to reoorganize it to be able to use for train, test, and validation

In [ ]:
# Assuming df is your metadata DataFrame and dataset_path is the path to your images
# Create a new directory to hold the organized dataset
organized_dataset_path = os.path.join("/kaggle/working", "organized_skin_cancer_dataset")
os.makedirs(organized_dataset_path, exist_ok=True)

# Get unique classes from your metadata
classes = df['dx'].unique()

# Create subfolders for each class in the new directory
for class_name in classes:
    class_dir = os.path.join(organized_dataset_path, class_name)
    os.makedirs(class_dir, exist_ok=True)

# Move images to their respective class folders
for index, row in df.iterrows():
    img_path = row['filepath']  # Path to the image
    class_name = row['dx']  # Class label
    destination_path = os.path.join(organized_dataset_path, class_name, os.path.basename(img_path))
    shutil.copy(img_path, destination_path)  # Copy the image to the new location

# Now, update your dataset_path to the new organized directory
dataset_path = organized_dataset_path

In [ ]:
dataset_path

# Dividing The Train, Val, Test in dataset

In [ ]:
# Read datasets
train_data = torchvision.datasets.ImageFolder(dataset_path, transform=train_transforms)
val_data = torchvision.datasets.ImageFolder(dataset_path, transform=eval_transform)
test_set = torchvision.datasets.ImageFolder(dataset_path, transform=eval_transform)

In [ ]:
# Split the original train data into training (80%) and validation (20%)
# Random split (only splitting indices, datasets are independent)
train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_indices, val_indices = torch.utils.data.random_split(range(len(train_data)),
                                                          [train_size, val_size])
# train_data and val_data have different transforms
train_set = torch.utils.data.Subset(train_data, train_indices)
val_set = torch.utils.data.Subset(val_data, val_indices)

In [ ]:
# Define the data loaders for the training, validation, and test sets
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_set, batch_size=32, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False)

# Model

In [ ]:
# get a pretrain EfficientNet-B0
#model = torchvision.models.resnet18(weights='IMAGENET1K_V1')
# --- EfficientNet-B0 backbone with ImageNet-1K weights ---

# Load pretrained EfficientNet-B0
model = models.efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)

# (Optional) adjust dropout (default is 0.2 at classifier[0])
# model.classifier[0].p = 0.2  # leave as-is unless you want to tune

In [ ]:
# Add a new layer/change the last layer
#model.fc = nn.Linear(model.fc.in_features, num_class)
# Replace classifier head (EfficientNet-B0 classifier is [Dropout, Linear])
in_features = model.classifier[1].in_features  # 1280 for B0
model.classifier[1] = nn.Linear(in_features, num_class)

In [ ]:
print(model)

In [ ]:
import torch.optim as optim

# Defined Cross-Entropy Loss
criteria = nn.CrossEntropyLoss()

# Defined Optimizer (Adam with learning rate 1e-4)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Getting Read of Training the Model

In [ ]:
# a function for training one epoch
def train_one_epoch(model, dataloader):
  # Prepare for storing loss and accuracy
  losses = MeanMetric().to(device)
  acc = Accuracy(task='multiclass', num_classes=10).to(device)
  model.train() # set model to train mode
  # a loop to iterate input(X) and label(Y) for all mini-batches
  for X, Y in tqdm(dataloader):
    X = X.to(device)
    Y = Y.to(device)
    optimizer.zero_grad() # reset optimizer
    preds = model(X) # model forward
    loss = criteria(preds, Y) # calculate loss
    loss.backward() # compute gradients via backpropagation
    optimizer.step() # perform gradient descent
    preds = preds.argmax(dim=1) # obtain the final predicted class
    losses.update(loss, X.size(0)) # store loss per batch
    acc.update(preds, Y) # store accuracy per batch
  return losses.compute().item(), acc.compute().item()

In [ ]:
# a function for validation one epoch
def validation_one_epoch(model, dataloader):
  # Prepare for storing loss and accuracy
  losses = MeanMetric().to(device)
  acc = Accuracy(task='multiclass', num_classes=10).to(device)
  model.eval() # set model to validation mode
  with torch.no_grad(): # disables gradient computation for evaluation
    # a loop to iterate input(X) and label(Y) for all mini-batches
    for X, Y in tqdm(dataloader):
      X = X.to(device)
      Y = Y.to(device)
      preds = model(X) # model forward
      loss = criteria(preds, Y) # calculate loss
      preds = preds.argmax(dim=1) # obtain the final predicted class
      losses.update(loss, X.size(0)) # store loss per batch
      acc.update(preds, Y) # store accuracy per batch
  return losses.compute().item(), acc.compute().item()

In [ ]:
# Prepare for storing loss and accuracy
best_val_loss = float('inf')  # Initialize best_val_loss to infinity
history = pd.DataFrame() # store statics for each epoch
epochs = 20 # number of epochs
# a loop for epochs
for i in range(0, epochs):
  # train one epoch
  train_loss, train_acc = train_one_epoch(model, train_dataloader)
  # validation one epoch
  val_loss, val_acc = validation_one_epoch(model, val_dataloader)
  # store and print loss and accuracy per epoch
  statistics = pd.DataFrame({
      "epoch": [i],   "train_loss": [train_loss],
                      "train_acc": [train_acc],
                      "val_loss": [val_loss],
                      "val_acc": [val_acc]})
  history = pd.concat([history, statistics], ignore_index=True)
  print(statistics.to_dict(orient="records")[0])

In [ ]:
# import os, torch
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"   # force sync for clearer traceback

# sanity check one batch from each loader
# def sanity_check(loader, model):
#     model.eval()
#     xb, yb = next(iter(loader))
#     print("xb dtype/shape:", xb.dtype, xb.shape)
#     print("yb dtype/shape:", yb.dtype, yb.shape, "min/max:", int(yb.min()), int(yb.max()))
#     with torch.no_grad():
#         logits = model(xb.to(next(model.parameters()).device))
#     print("logits shape:", logits.shape)
#     num_classes = logits.shape[1]
#     print("num_classes (from model):", num_classes)
#     assert yb.min().item() >= 0, "Found negative labels"
#     assert yb.max().item() < num_classes, f"Label {int(yb.max())} >= num_classes {num_classes}"

# sanity_check(train_dataloader, model)
# sanity_check(val_dataloader, model)

# Model Evaluation

In [ ]:
# Create a figure with two subplots (side by side)
plt.figure(figsize=(8, 3))
# Plot Loss Curve (Train + Validation)
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first plot
plt.plot(history["epoch"], history["train_loss"], label="Train", color="blue")
plt.plot(history["epoch"], history["val_loss"], label="Validation", color="red")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
# Plot Accuracy Curve (Train + Validation)
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second plot
plt.plot(history["epoch"], history["train_acc"], label="Train", color="blue")
plt.plot(history["epoch"], history["val_acc"], label="Validation", color="red")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
# Adjust layout and show the plots
plt.tight_layout()
plt.show()

In [ ]:
# prepare for storing evaluation metrics
test_acc = Accuracy(task='multiclass', num_classes=num_class).to(device)
test_confusion_matrix=ConfusionMatrix(task="multiclass", num_classes=num_class).to(device)
test_precision = Precision(task="multiclass", num_classes=num_class, average="macro").to(device)
test_recall = Recall(task="multiclass", num_classes=num_class, average="macro").to(device)
test_f1_score = F1Score(task="multiclass", num_classes=num_class, average="macro").to(device)

model = model.to(device)
model.eval() # set model to evaluation mode
with torch.no_grad():
  for X, Y in test_dataloader:
    X = X.to(device)
    Y = Y.to(device)
    preds = model(X) # model forward
    preds = preds.argmax(dim=1) # obtain the final predicted class
    # store loss and accuracy per batc
    test_confusion_matrix.update(preds, Y)
    test_acc.update(preds, Y)
    test_precision.update(preds, Y)
    test_recall.update(preds, Y)
    test_f1_score.update(preds, Y)
  # Print the results
  print("Confusion Matrix:\n", test_confusion_matrix.compute())
  print("Accuracy:", test_acc.compute().item())
  print("Precision:", test_precision.compute().item())
  print("Recall:", test_recall.compute().item())
  print("F1 Score:", test_f1_score.compute().item())

In [ ]:
# Create a heatmap for better confusion matrix visualization
sns.heatmap(test_confusion_matrix.compute().cpu(), annot=True, fmt="d",
            cmap="Blues", xticklabels=classes, yticklabels=classes)
# Labels and title
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

**simple Grad-CAM**

In [ ]:
import cv2

# Function to generate Grad-CAM
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer

        self.gradients = None
        self.activations = None

        # Hook to capture gradients
        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations = output

    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]

    def generate(self, input_image, target_class=None):
        self.model.eval()
        output = self.model(input_image)

        if target_class is None:
            target_class = output.argmax(dim=1)

        loss = output[0, target_class]
        self.model.zero_grad()
        loss.backward()

        gradients = self.gradients[0].cpu().data.numpy()
        activations = self.activations[0].cpu().data.numpy()

        weights = np.mean(gradients, axis=(1, 2))
        cam = np.zeros(activations.shape[1:], dtype=np.float32)

        for i, w in enumerate(weights):
            cam += w * activations[i]

        cam = np.maximum(cam, 0)
        cam = cv2.resize(cam, (224, 224))
        cam = cam - np.min(cam)
        cam = cam / np.max(cam)
        return cam

# Pick a sample from test set
sample_img, sample_label = next(iter(test_dataloader))
sample_img = sample_img[0].unsqueeze(0).to(device)  # take one image

# Apply Grad-CAM
target_layer = model.layer4[-1]  # usually last layer for ResNet18
gradcam = GradCAM(model, target_layer)
cam = gradcam.generate(sample_img)

# Plot original + heatmap
img = sample_img.cpu().squeeze().permute(1,2,0).numpy()
img = (img - img.min()) / (img.max() - img.min())  # normalize image

plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.title("Original Image")
plt.imshow(img)
plt.axis('off')

plt.subplot(1,2,2)
plt.title("Grad-CAM Heatmap")
plt.imshow(img)
plt.imshow(cam, cmap='jet', alpha=0.5)  # overlay Grad-CAM
plt.axis('off')
plt.show()

# Saving The Model

In [ ]:
# To save the best model
best_val_loss = float('inf')  # Initialize best_val_loss to infinity

if val_loss < best_val_loss:
    best_val_loss = val_loss
    torch.save(model.state_dict(), "best_model6.pth")
    print("Best model saved!")

## 🔧 Step 1: Hyperparameter Tuning

We'll explore different hyperparameter settings to improve model performance:
- Optimizer: Try switching from Adam to AdamW
- Learning Rate: Test different values (1e-3, 1e-4, 5e-5)
- Scheduler: Add ReduceLROnPlateau to adjust learning rate dynamically
- Epochs: Increase from default (if low)
- Batch Size: Experiment with smaller/larger values


In [ ]:
# Define optimizer and learning rate scheduler
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

learning_rate = 1e-4  # Try: 1e-3, 5e-5
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-2)

# Scheduler that reduces LR when a metric has stopped improving
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5, verbose=True)


In [ ]:
# Modified training loop to include scheduler step
train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss, train_acc = train_one_epoch(model, train_dataloader)
    val_loss, val_acc = validation_one_epoch(model, test_dataloader)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    # Step the scheduler
    scheduler.step(val_loss)

    print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")
